In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
img_height = 300
img_width = 200
batch_size = 32

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Przykładowe wartości
])

data_dir = './raw-img'  # Path to your dataset
dataset = ImageFolder(root=data_dir, transform=transform)

classes = dataset.classes

dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_data, val_data= random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

cpu


In [38]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, batch_size, kernel_size=11, padding=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=batch_size, out_channels=64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=96, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # tutaj nie dawać dropout
            nn.Flatten(),
        )

        n_channels = self.feature_extractor(torch.empty(1, 3, img_height, img_width)).size(-1)
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features=n_channels, out_features=1024, bias=1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features=n_channels, out_features=1024, bias=1),
            nn.ReLU(inplace=True),
            
            nn.Linear(in_features=1024, out_features=len(classes), bias=1)
        )


    def forward(self, x):
        features = self.feature_extractor(x)
        out = self.classifier(features)
        return out


net = SimpleCNN()

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# uczenie


net.train()
for epoch in range(1):  # liczba epok
    loss_epoch = 0
    total_correct = 0
    total_samples = 0
    
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss_epoch += loss.item()
        loss.backward()
        optimizer.step()
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
    accuracy = 100 * total_correct / total_samples
        # print(f"epoch {epoch}, batch = {i}, loss = {loss.item()}, inputs.shape = {inputs.shape}")
    print(f"epoch = {epoch}, accuracy = {accuracy}, loss = {loss_epoch}")

In [9]:
# testowanie
net.eval()

outputs_all = []
labels_all = []
with torch.no_grad():
    for batch in test_loader:
        images, labels = batch
        outputs = net(images)
        outputs_all.append(torch.softmax(outputs, dim=1).detach().cpu().numpy())
        labels_all.append(labels.numpy())

outputs_all = np.concatenate(outputs_all)
labels_all = np.concatenate(labels_all)

print(outputs_all)
print(labels_all)

print(outputs_all.shape)
print(labels_all.shape)

[[6.1840727e-04 2.4415588e-04 8.3972716e-01 ... 3.1816293e-03
  7.1498338e-04 6.6428341e-02]
 [5.7939583e-01 2.9078059e-02 2.5012193e-02 ... 3.7561566e-02
  2.4268913e-01 3.7238125e-03]
 [3.0388923e-03 6.7567853e-03 3.1161809e-01 ... 2.1343412e-02
  2.6483095e-01 9.2413835e-02]
 ...
 [1.7165463e-02 1.4238151e-02 7.9960957e-02 ... 5.3411812e-02
  4.7253884e-02 7.3325986e-01]
 [1.2043575e-03 1.4729572e-02 2.1929260e-01 ... 3.0278873e-01
  9.9355496e-02 1.1293743e-01]
 [2.9673523e-03 1.4596226e-02 1.3253026e-02 ... 1.4334393e-01
  7.2955735e-02 3.6979888e-02]]
[6 2 2 ... 2 7 2]
(5236, 10)
(5236,)


In [91]:
from PIL import Image
image_path = './val_img/butterfly.jpg'
image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0)

with torch.no_grad():  # No gradient computation for inference
    outputs = net(image_tensor)
    _, predicted_class = torch.max(outputs, 1)  # Get the class index with the highest score


predicted_label = classes[predicted_class.item()]

print(f"Predicted Class: {predicted_label}")


Predicted Class: squirrel


In [ ]:

AUC = roc_auc_score(y_true=labels_all, y_score=outputs_all[:,1])
print(f"AUC = {AUC}")

fpr, tpr, thresholds = roc_curve(y_true=labels_all, y_score=outputs_all[:,1], pos_label=1)
plt.plot(fpr, tpr)


y_d = np.argmax(outputs_all, axis=1)
tn, fp, fn, tp = confusion_matrix(y_true=labels_all, y_pred=y_d).ravel()

print(f"tn {tn}, fp {fp}, fn {fn}, tp {tp}")

# _ = ConfusionMatrixDisplay.from_estimator(classifier_05, X_test, y_test)

def get_label(x):
    return ['benign' if z == 0 else 'malignant' for z in x]

cm = confusion_matrix(y_true=get_label(labels_all), y_pred=get_label(y_d), labels=['benign','malignant'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['benign','malignant'])
disp.plot()
plt.show()

In [10]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: butterfly is 63.6 %
Accuracy for class: cat   is 20.1 %
Accuracy for class: chicken is 61.0 %
Accuracy for class: cow   is 26.8 %
Accuracy for class: dog   is 45.6 %
Accuracy for class: elephant is 37.8 %
Accuracy for class: horse is 58.5 %
Accuracy for class: sheep is 46.0 %
Accuracy for class: spider is 85.2 %
Accuracy for class: squirrel is 55.0 %
